<a href="https://colab.research.google.com/github/ILMNX/Pattern-recognition/blob/main/text-classification-LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences # Menyelaraskan panjang input data teks dengan padding, agar semua input
# memiliki panjang yang seragam.

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential # Digunakan untuk membuat model urutan yang terdiri dari layer berturut-turut.

from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense, Bidirectional
 # embeding : Layer pertama dalam model untuk memetakan kata ke dalam representasi vektor berdimensi rendah.
 # LSTM : Merupakan tipe RNN (Recurrent Neural Network) yang digunakan untuk memproses data urutan, misalnya teks.
 # Dropout : Digunakan untuk regularisasi model dengan membuang (drop) neuron secara acak selama pelatihan, untuk mencegah overfitting.
 # Dense :  Layer yang sepenuhnya terhubung (fully connected layer), biasanya digunakan untuk output akhir dari model.
 # Bidirectional : Membungkus LSTM atau GRU untuk memproses data dalam dua arah (forward dan backward).

from tensorflow.keras.callbacks import EarlyStopping
# EarlyStooping : Digunakan untuk menghentikan pelatihan model secara otomatis jika akurasi model tidak
# meningkat setelah sejumlah epoch tertentu, untuk mencegah overfitting.


In [ ]:
texts = ["I love programming", "I hate bugs", "Deep learning is amazing", "I dislike errors",
         "This is a great product", "Terrible service", "The food was delicious", "The movie was boring",
         "I am very happy", "I am very sad"] * 100 # Perbanyak data
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0] * 100

Penjelasan :     

* texts menjadi list berisi 1000 kalimat yang mencerminkan opini atau perasaan.
labels menjadi list berisi 1000 label yang menunjukkan apakah kalimat tersebut bersifat positif (1) atau negatif (0).
* Tujuan
texts dan labels ini biasanya digunakan dalam pelatihan model klasifikasi teks, di mana model akan mempelajari pola dalam kalimat (dari texts) dan memprediksi labelnya (positif atau negatif) berdasarkan data latih tersebut.

In [ ]:
# Tokenisasi dan padding
max_words = 10000
max_sequence_length = 100
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=max_sequence_length)
y = np.array(labels)


Penjelasan :   
Kode ini melakukan tokenisasi dan padding pada data teks. Pertama, **`Tokenizer`** digunakan untuk mengonversi teks menjadi urutan angka dengan membatasi jumlah kata hingga **10.000** dan menyertakan token untuk kata yang tidak dikenal (**`oov_token="<OOV>"`**). Kemudian, **`fit_on_texts(texts)`** melatih tokenizer berdasarkan data teks, dan **`texts_to_sequences(texts)`** mengonversi teks menjadi urutan angka. Setelah itu, **`pad_sequences`** digunakan untuk menyamakan panjang setiap urutan teks hingga **100 kata**. Data teks yang sudah diproses disimpan di **`X`**, sementara labelnya disimpan di **`y`** sebagai array NumPy.

In [ ]:
# Bagi dataset menjadi data pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:

# Langkah 2: Membangun Model LSTM

embedding_dim = 128  # Dimensi embedding

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(64, return_sequences=True))) # Bidirectional LSTM
model.add(Dropout(0.2)) # Tambah dropout
model.add(Bidirectional(LSTM(32))) # Bidirectional LSTM
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2)) # Tambah dropout
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Penjelasan :      

  Kode ini membangun model **LSTM** untuk klasifikasi teks. Pertama, **`Embedding`** digunakan untuk mengonversi kata menjadi vektor berdimensi **128**. Kemudian, dua **Bidirectional LSTM** ditambahkan, yang memungkinkan model memproses data dari kedua arah (maju dan mundur), dengan unit LSTM pertama berukuran 64 dan yang kedua berukuran 32. **`Dropout`** dengan rate 0.2 digunakan untuk mencegah overfitting. Setelah itu, layer **`Dense`** dengan 16 neuron dan fungsi aktivasi **ReLU** ditambahkan untuk pemrosesan lebih lanjut, diikuti dengan **`Dropout`** lagi. Terakhir, layer **`Dense`** dengan 1 neuron dan aktivasi **sigmoid** digunakan untuk menghasilkan output biner (0 atau 1), yang cocok untuk klasifikasi dua kelas.

In [ ]:

# Langkah 3: Kompilasi dan Pelatihan Model

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True) # Early stopping
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 15s 379ms/step - accuracy: 0.5216 - loss: 0.6856 - val_accuracy: 0.6625 - val_loss: 0.6227
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 463ms/step - accuracy: 0.7502 - loss: 0.5530 - val_accuracy: 1.0000 - val_loss: 0.2280
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 293ms/step - accuracy: 0.9835 - loss: 0.1949 - val_accuracy: 1.0000 - val_loss: 0.0645
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 379ms/step - accuracy: 1.0000 - loss: 0.1011 - val_accuracy: 1.0000 - val_loss: 0.0264
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 295ms/step - accuracy: 1.0000 - loss: 0.0706 - val_accuracy: 1.0000 - val_loss: 0.0118
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 354ms/step - accuracy: 1.0000 - loss: 0.0493 - val_accuracy: 1.0000 - val_loss: 0.0054
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 288ms/step - accuracy: 1.0000 - loss: 0.0343 - val_accuracy: 1.0000 - val_loss: 0.0027
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 291ms/step - accuracy: 1.0000 - loss: 0.0211 - val_accuracy: 1

Penjelasan :     

Kode ini mengompilasi dan melatih model LSTM. **`model.compile`** mengonfigurasi model dengan fungsi **loss** `binary_crossentropy` (untuk klasifikasi biner), **optimizer** `adam`, dan metrik **`accuracy`** untuk evaluasi. **`EarlyStopping`** digunakan untuk menghentikan pelatihan jika **loss validasi** tidak membaik setelah 3 epoch, sambil memulihkan bobot model terbaik (dengan `restore_best_weights=True`). Kemudian, **`model.fit`** melatih model menggunakan data latih **`X_train`** dan **`y_train`**, dengan 20 epoch, ukuran batch 64, dan 10% data digunakan untuk validasi (**`validation_split=0.1`**).

In [ ]:

# Langkah 4: Evaluasi Model

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 1.0000 - loss: 6.8832e-05
Loss: 0.0001
Accuracy: 1.0000


Penjelasan :     

 Kode ini mengevaluasi performa model pada data uji **`X_test`** dan **`y_test`** menggunakan **`model.evaluate`**. Hasil evaluasi, yaitu **loss** dan **accuracy**, disimpan dalam variabel **`loss`** dan **`accuracy`**, kemudian dicetak dengan format desimal 4 angka setelah koma.

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 100, 128)            │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100, 128)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 64)                  │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,263,269 (16.26 MB)

 Trainable params: 1,421,089 (5.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,842,180 (10.84 MB)

Penjelasan :     


Hasil di atas menunjukkan arsitektur model LSTM yang digunakan. Berikut adalah penjelasannya:

1. **`Embedding` Layer**: Mengubah kata-kata menjadi vektor berdimensi 128. Memiliki 1.280.000 parameter (berdasarkan ukuran vocabulary dan embedding dimension).
2. **`Bidirectional LSTM` Layer**: Memproses data teks dari dua arah (maju dan mundur) dengan output berukuran 128 dan 98.816 parameter.
3. **`Dropout` Layer**: Mencegah overfitting dengan mengatur sebagian neuron menjadi 0 selama pelatihan, tanpa menambah parameter.
4. **`Bidirectional LSTM` Layer**: Layer LSTM kedua dengan output 64 dan 41.216 parameter.
5. **`Dense` Layer**: Layer dengan 16 neuron dan 1.040 parameter.
6. **`Dropout` Layer**: Lagi, mencegah overfitting tanpa menambah parameter.
7. **`Dense` Layer**: Output layer dengan 1 neuron untuk klasifikasi biner, dengan 17 parameter.

Total parameter model: **4.263.269**, dengan **1.421.089** parameter yang dapat dilatih. Model ini membutuhkan sekitar 16.26 MB memori untuk disimpan.